# Silver-zu-Gold Agent Flow

## Überblick
Dieses Notebook demonstriert den **Data Mart Erstellungsprozess** der Agentic ELT Pipeline.
Der Gold Layer transformiert bereinigte Silver-Daten in business-ready Analytics-Tabellen.

### Agent-Workflow:
1. **Gold Draft Agent** → Analysiert Silver-Daten und entwirft Star-Schema
2. **Gold Builder Agent** → Generiert Data Mart Code
3. **Gold Runner** → Erstellt Dimensions-, Fact- und Aggregat-Tabellen

### Was Sie lernen:
- Wie LLM-Agents Star-Schemas für Analytics entwerfen
- Welche Business-Metriken und KPIs automatisch identifiziert werden
- Wie Agents optimierte Data Marts für BI-Tools erstellen

## 📋 Setup und Konfiguration

**Was passiert hier:**
- Laden der benötigten Python-Bibliotheken
- Definition der Pfade zu Silver-, Gold- und Draft-Report-Artefakten
- Überprüfung der Datenverfügbarkeit über verschiedene Pipeline-Stufen

In [ ]:
import json
import pandas as pd
from pathlib import Path
import yaml

# Setzen Sie hier Ihre Silver und Gold Run-IDs
# Hinweis: Gold nutzt Silver run_id als Input, erstellt aber Marts unter derselben run_id
silver_run_id = "20260124_231151_#6470e523"  # Aktualisieren Sie dies für Ihren Silver-Lauf
gold_run_id = "20260124_231016_#6470e523"    # Gold Marts nutzen Bronze run_id für Konsistenz

# Pfade definieren
silver_dir = Path(f"../artifacts/silver/{silver_run_id}")
gold_dir = Path(f"../artifacts/gold/marts/{gold_run_id}")
draft_reports_dir = Path(f"../tmp/draft_reports/gold/{gold_run_id}")

print(f"🔍 Überprüfe Datenverzeichnisse:")
print(f"Silver Directory: {silver_dir} ({'✅ vorhanden' if silver_dir.exists() else '❌ fehlt'})")
print(f"Gold Directory: {gold_dir} ({'✅ vorhanden' if gold_dir.exists() else '❌ fehlt'})")
print(f"Draft Reports: {draft_reports_dir} ({'✅ vorhanden' if draft_reports_dir.exists() else '❌ fehlt'})")

## 🥈 Silver Layer Analyse

**Was passiert hier:**
- Analyse der bereinigten Silver-Daten, die als Input für das Gold Layer Design dienen
- Überprüfung des Silver Agent Context und Performance-Metriken
- Kategorisierung der Tabellen nach Business-Domänen (Kunden, Produkte, Verkäufe, Standorte)

**Warum wichtig:**
Der Gold Draft Agent nutzt diese Silver Layer Analyse, um verfügbare Datenstrukturen zu verstehen und angemessene Star-Schema Dimensionen und Facts zu entwerfen.

In [ ]:
# Silver Agent Context analysieren
silver_context_path = silver_dir / "reports" / "silver_run_agent_context.json"
if silver_context_path.exists():
    with open(silver_context_path, 'r') as f:
        silver_context = json.load(f)
    
    print("📊 Silver Context für Gold Planung:")
    print(f"Run ID: {silver_context.get('run_id')}")
    print(f"Dateien verarbeitet: {silver_context.get('files_total')}")
    
    # Performance Metriken
    perf = silver_context.get('performance_metrics', {})
    print(f"\n📈 Silver Layer Performance:")
    print(f"  Zeilen gesamt: {perf.get('total_rows_processed', 0):,}")
    print(f"  Spalten gesamt: {perf.get('total_columns_processed', 0):,}")
    print(f"  Tabellen verarbeitet: {perf.get('tables_processed', 0)}")
    print(f"  Tabellen fehlgeschlagen: {perf.get('tables_failed', 0)}")
else:
    print("❌ Silver Agent Context nicht gefunden")

In [ ]:
# Silver Tabellen für Star-Schema Design kategorisieren
silver_data_dir = silver_dir / "data"
if silver_data_dir.exists():
    csv_files = list(silver_data_dir.glob("*.csv"))
    
    print(f"🗂️ Silver Tabellen für Gold Design ({len(csv_files)} Tabellen):")
    
    # Tabellen nach Business-Domäne kategorisieren
    customer_tables = []
    product_tables = []
    sales_tables = []
    location_tables = []
    other_tables = []
    
    for csv_file in csv_files:
        name = csv_file.name.lower()
        if 'cst' in name or 'customer' in name:
            customer_tables.append(csv_file.name)
        elif 'prd' in name or 'product' in name:
            product_tables.append(csv_file.name)
        elif 'sales' in name:
            sales_tables.append(csv_file.name)
        elif 'loc' in name or 'location' in name:
            location_tables.append(csv_file.name)
        else:
            other_tables.append(csv_file.name)
    
    print(f"\n🏗️ Star-Schema Design Kategorien:")
    print(f"  Kunden-Dimension Quellen: {customer_tables}")
    print(f"  Produkt-Dimension Quellen: {product_tables}")
    print(f"  Standort-Dimension Quellen: {location_tables}")
    print(f"  Verkaufs-Fact Quellen: {sales_tables}")
    print(f"  Andere Tabellen: {other_tables}")
else:
    print("❌ Silver Daten nicht gefunden")

## 🤖 Gold Draft Agent - Star-Schema Design

**Was passiert hier:**
- Überprüfung des LLM-generierten Star-Schema Designs vom Gold Draft Agent
- Analyse der vorgeschlagenen Dimensionen, Facts und Business-Metriken
- Verstehen der Agent-Begründung für die Data Mart Architektur

**Agent-Logik:**
Der Gold Draft Agent analysiert Silver-Daten um:
- Dimensionstabellen für Stammdaten zu entwerfen (Kunden, Produkte, Standorte)
- Fact-Tabellen für Transaktionsdaten zu erstellen (Verkäufe, Bestellungen)
- Aggregationsmöglichkeiten für Performance zu identifizieren
- Wide Tables für Ad-hoc Analytics zu planen

In [ ]:
# Gold Design Report analysieren
gold_design_path = draft_reports_dir / "gold_run_human_report.md"
if gold_design_path.exists():
    with open(gold_design_path, 'r', encoding='utf-8') as f:
        gold_design = f.read()
    
    print("📄 Gold Draft Agent - Design Report:")
    print("=" * 60)
    
    # Zeige ersten Teil des Design Reports
    lines = gold_design.split('\n')
    for i, line in enumerate(lines[:35]):  # Erste 35 Zeilen
        print(line)
    
    if len(lines) > 35:
        print(f"\n... ({len(lines)-35} weitere Zeilen)")
    
    print(f"\n📊 Design Report Statistiken:")
    print(f"  Gesamtlänge: {len(gold_design):,} Zeichen")
    print(f"  Zeilen: {len(lines)}")
    print(f"  Abschnitte: {gold_design.count('#')} (geschätzt)")
else:
    print("❌ Gold Design Report nicht gefunden")

## 📋 Gold Mart Plan - LLM Architektur

**Was passiert hier:**
- Inspektion des strukturierten JSON Mart Plans vom Gold Draft Agent
- Überprüfung der geplanten Dimensionen, Facts und Data Mart Spezifikationen
- Verstehen der Agent-Priorisierung und Risikobewertung

**Mart Plan Struktur:**
Der Gold Draft Agent erstellt einen detaillierten Plan, den der Gold Builder Agent verwendet, um ausführbaren Code für die Data Mart Erstellung zu generieren.

In [ ]:
# Gold Agent Context (Mart Plan) analysieren
gold_context_path = draft_reports_dir / "gold_run_agent_context.json"
if gold_context_path.exists():
    with open(gold_context_path, 'r') as f:
        gold_context = json.load(f)
    
    print("🤖 Gold Draft Agent - Mart Plan:")
    print(f"Zielsetzung: {gold_context.get('gold_layer_objective', 'N/A')}")
    
    # Dimensionen Analyse
    dimensions = gold_context.get('dimensions', [])
    print(f"\n🏛️ Geplante Dimensionen ({len(dimensions)}):")
    for dim in dimensions:
        print(f"  📊 {dim.get('name')}: {dim.get('grain')}")
        print(f"    Schlüssel: {dim.get('primary_keys')}")
        attrs = dim.get('attributes', [])
        print(f"    Attribute: {attrs[:3]}{'...' if len(attrs) > 3 else ''}")
        print(f"    SCD: {dim.get('scd_guidance', 'N/A')}")
    
    # Facts Analyse
    facts = gold_context.get('facts', [])
    print(f"\n📈 Geplante Facts ({len(facts)}):")
    for fact in facts:
        print(f"  📊 {fact.get('name')}: {fact.get('grain')}")
        print(f"    Schlüssel: {fact.get('primary_keys')}")
        measures = fact.get('measures', [])
        print(f"    Kennzahlen: {measures[:3]}{'...' if len(measures) > 3 else ''}")
        agg_ops = fact.get('aggregation_opportunities', [])
        print(f"    Aggregationen: {len(agg_ops)} Möglichkeiten")
    
    # Marts Zusammenfassung
    marts = gold_context.get('marts', [])
    mandatory_marts = [m for m in marts if m.get('priority') == 'mandatory']
    optional_marts = [m for m in marts if m.get('priority') == 'optional']
    
    print(f"\n🏪 Geplante Data Marts ({len(marts)} gesamt):")
    print(f"  Pflicht: {len(mandatory_marts)} Marts")
    print(f"  Optional: {len(optional_marts)} Marts")
    
    for mart in mandatory_marts:
        print(f"    ✅ {mart.get('name')} ({mart.get('type')})")
    
    # Risikobewertung
    risks = gold_context.get('risks', [])
    next_steps = gold_context.get('next_steps', [])
    print(f"\n⚠️ Risikobewertung: {len(risks)} Risiken identifiziert")
    print(f"📋 Implementierungsplan: {len(next_steps)} Schritte definiert")
else:
    print("❌ Gold Agent Context nicht gefunden")

## 🥇 Gold Data Marts Analyse

**Was passiert hier:**
- Inspektion der erstellten Gold Data Marts und ihrer Struktur
- Kategorisierung der Tabellen nach Typ (Dimensionen, Facts, Aggregate, Wide Tables)
- Analyse des implementierten Star-Schemas und der Datenbeziehungen

**Data Mart Typen:**
Der Gold Runner erstellt verschiedene Tabellentypen, die für verschiedene Analytics-Anwendungsfälle optimiert sind.

In [ ]:
# Gold Data Marts analysieren
gold_data_dir = gold_dir / "data"
if gold_data_dir.exists():
    gold_csv_files = list(gold_data_dir.glob("*.csv"))
    
    print(f"🥇 Gold Data Marts ({len(gold_csv_files)} Tabellen):")
    
    # Gold Tabellen nach Typ kategorisieren
    dimensions = []
    facts = []
    aggregates = []
    wide_tables = []
    other_tables = []
    
    for csv_file in gold_csv_files:
        name = csv_file.name.lower()
        if 'dim_' in name:
            dimensions.append(csv_file.name)
        elif 'fact_' in name:
            facts.append(csv_file.name)
        elif 'agg_' in name:
            aggregates.append(csv_file.name)
        elif 'wide_' in name:
            wide_tables.append(csv_file.name)
        else:
            other_tables.append(csv_file.name)
    
    print(f"\n🏗️ Star-Schema Implementierung:")
    print(f"  🏛️ Dimensionen: {dimensions}")
    print(f"  📈 Facts: {facts}")
    print(f"  📊 Aggregate: {aggregates}")
    print(f"  📋 Wide Tables: {wide_tables}")
    print(f"  ❓ Andere: {other_tables}")
    
    # Detaillierte Tabellenanalyse
    print(f"\n📊 Detaillierte Tabellenanalyse:")
    for csv_file in gold_csv_files:
        df = pd.read_csv(csv_file)
        table_type = "📊 Andere"
        if csv_file.name in dimensions:
            table_type = "🏛️ Dimension"
        elif csv_file.name in facts:
            table_type = "📈 Fact"
        elif csv_file.name in aggregates:
            table_type = "📊 Aggregat"
        elif csv_file.name in wide_tables:
            table_type = "📋 Wide Table"
        
        print(f"\n  {table_type} {csv_file.name}:")
        print(f"    Dimensionen: {df.shape[0]:,} Zeilen × {df.shape[1]} Spalten")
        print(f"    Spalten: {list(df.columns)}")
        
        if len(df) > 0:
            # Zeige Beispiel-Datenstruktur
            sample_row = df.iloc[0].to_dict()
            print(f"    Beispiel: {list(sample_row.keys())[:5]}{'...' if len(sample_row) > 5 else ''}")
else:
    print("❌ Gold Daten nicht gefunden")

## 📈 Business KPIs und Metriken

**Was passiert hier:**
- Analyse der verfügbaren Business-Metriken im Gold Layer
- Berechnung von Beispiel-KPIs aus Fact-Tabellen
- Identifikation von dimensionalen Analysemöglichkeiten

**KPI-Analyse:**
Demonstration, wie der Gold Layer Business Intelligence und Analytics ermöglicht, indem er gebrauchsfertige Metriken bereitstellt.

In [ ]:
# Business KPIs aus Fact-Tabellen analysieren
if gold_data_dir.exists():
    fact_files = [f for f in gold_csv_files if 'fact_' in f.name.lower()]
    
    print(f"📈 BUSINESS KPI ANALYSE")
    print("=" * 50)
    
    for fact_file in fact_files:
        df_fact = pd.read_csv(gold_data_dir / fact_file.name)
        print(f"\n📊 KPI-Analyse für {fact_file.name}:")
        
        # Numerische Spalten für Metriken identifizieren
        numeric_cols = df_fact.select_dtypes(include=['number']).columns.tolist()
        print(f"  Verfügbare Metriken: {numeric_cols}")
        
        # Beispiel-KPIs berechnen
        if len(numeric_cols) > 0:
            for col in numeric_cols[:3]:  # Erste 3 numerische Spalten
                if df_fact[col].notna().any():
                    print(f"\n  📊 {col} Statistiken:")
                    print(f"    Gesamt: {df_fact[col].sum():,.2f}")
                    print(f"    Durchschnitt: {df_fact[col].mean():,.2f}")
                    print(f"    Minimum: {df_fact[col].min():,.2f}")
                    print(f"    Maximum: {df_fact[col].max():,.2f}")
        
        # Dimensionale Analysemöglichkeiten
        date_cols = [col for col in df_fact.columns if 'date' in col.lower() or 'time' in col.lower()]
        id_cols = [col for col in df_fact.columns if 'id' in col.lower() and col.lower() != 'id']
        
        if date_cols:
            print(f"\n  📅 Zeit-Dimensionen: {date_cols}")
        
        if id_cols:
            print(f"  🔗 Dimensionale Schlüssel: {id_cols}")
            for id_col in id_cols[:2]:  # Erste 2 ID-Spalten
                unique_count = df_fact[id_col].nunique()
                print(f"    {id_col}: {unique_count:,} eindeutige Werte")
else:
    print("❌ Kann KPIs nicht analysieren - Gold Daten nicht verfügbar")

## ✅ Data Mart Qualitätsbewertung

**Was passiert hier:**
- Überprüfung der Gold Layer Metadaten und Ausführungsergebnisse
- Bewertung der Data Mart Erstellungserfolgsraten
- Validierung der Star-Schema Implementierungsqualität

**Qualitätsmetriken:**
Bewertung des Erfolgs der Gold Layer Transformation und des Data Mart Erstellungsprozesses.

In [ ]:
# Gold Layer Metadaten analysieren
gold_metadata_path = gold_dir / "metadata.yaml"
if gold_metadata_path.exists():
    with open(gold_metadata_path, 'r') as f:
        gold_metadata = yaml.safe_load(f)
    
    print("📋 GOLD LAYER QUALITÄTSBEWERTUNG")
    print("=" * 50)
    
    print(f"\n📊 Ausführungszusammenfassung:")
    print(f"  Run ID: {gold_metadata.get('run_id')}")
    print(f"  Layer: {gold_metadata.get('layer')}")
    print(f"  Quell-Layer: {gold_metadata.get('source_layer')}")
    
    summary = gold_metadata.get('summary', {})
    print(f"\n📈 Verarbeitungsergebnisse:")
    print(f"  Dateien gesamt: {summary.get('files_total')}")
    print(f"  Dateien erfolgreich: {summary.get('files_success')}")
    print(f"  Dateien fehlgeschlagen: {summary.get('files_failed')}")
    
    # Erfolgsrate berechnen
    total_files = summary.get('files_total', 0)
    success_files = summary.get('files_success', 0)
    if total_files > 0:
        success_rate = (success_files / total_files) * 100
        print(f"  Erfolgsrate: {success_rate:.1f}%")
    
    # Tabellen-Level Ergebnisse
    tables = gold_metadata.get('tables', {})
    print(f"\n📊 Tabellen-Level Ergebnisse:")
    successful_tables = 0
    total_rows = 0
    
    for table_name, table_info in tables.items():
        status = table_info.get('status', 'UNKNOWN')
        rows = table_info.get('rows_out', 0)
        total_rows += rows
        
        status_icon = "✅" if status == "SUCCESS" else "❌"
        print(f"  {status_icon} {table_name}: {status} ({rows:,} Zeilen)")
        
        if status == "SUCCESS":
            successful_tables += 1
    
    print(f"\n🎯 Qualitätsmetriken:")
    print(f"  Erfolgreiche Tabellen: {successful_tables}/{len(tables)}")
    print(f"  Zeilen gesamt erstellt: {total_rows:,}")
    print(f"  Durchschnittliche Zeilen pro Tabelle: {total_rows/len(tables) if tables else 0:,.0f}")
else:
    print("❌ Gold Metadaten nicht gefunden")

## 🔄 Agent-Workflow Zusammenfassung

**Was Sie gelernt haben:**
Dieser Abschnitt fasst den gesamten Gold Agent Workflow zusammen und erklärt, wie LLM-Agents business-ready Data Marts aus bereinigten Silver-Daten erstellen.

In [ ]:
print("🤖 GOLD AGENT WORKFLOW - ZUSAMMENFASSUNG")
print("=" * 60)
print()
print("1️⃣ GOLD DRAFT AGENT (Architektur & Design):")
print("   🔍 Liest Silver Agent Context und Metadaten")
print("   📊 Analysiert verfügbare Tabellen und Schemas")
print("   🏗️ Entwirft Star-Schema (Dimensionen, Facts, Aggregate)")
print("   📈 Identifiziert Business-KPIs und Metriken")
print("   📄 Erstellt gold_run_human_report.md (Design-Begründung)")
print("   📋 Erstellt gold_run_agent_context.json (Mart-Plan)")
print()
print("2️⃣ GOLD BUILDER AGENT (Code-Generierung):")
print("   📖 Liest Gold Draft Context")
print("   🛠️ Generiert ausführbaren Python-Code für Data Marts")
print("   🏛️ Implementiert Dimensionstabellen (SCD Type 1/2)")
print("   📈 Implementiert Fact-Tabellen mit Kennzahlen")
print("   📊 Implementiert Aggregat-Tabellen für Performance")
print("   📜 Erstellt load_3_gold_layer_runner.py")
print()
print("3️⃣ GOLD RUNNER (Ausführung):")
print("   ▶️ Führt generierten Code aus")
print("   🔄 Transformiert Silver → Gold Data Marts")
print("   🏗️ Erstellt Star-Schema Tabellen")
print("   💾 Generiert business-ready CSV-Dateien")
print("   📋 Erstellt Metadaten und Reports")
print()
print("✅ ERGEBNIS: Business-ready Data Marts für BI/Analytics")
print("   🏛️ Dimensionen für Stammdaten")
print("   📈 Facts für transaktionale Metriken")
print("   📊 Aggregate für performance-optimierte KPIs")
print("   📋 Wide Tables für Ad-hoc Analysen")
print("   🔗 Optimiert für BI-Tools (Tableau, Power BI)")
print("   📊 Gebrauchsfertige Business-Metriken und KPIs")